# Project 2: Features, Galore: A Linear Regression Analysis on Home Features Predicting Sale Price
---

## Test Model and Kaggle Submission 

This section includes running the test data through the optimal Linear Regression model. The model used is the regular Linear Regression with all non multi-collinear features. Predicted home sale price values are determined for submission in the Ames Housing Kaggle Competition. 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import mean_squared_error
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [2]:
#Read in files 
Z_train=pd.read_csv('../datasets/z_train_final.csv')
Z_test_data=pd.read_csv('../datasets/clean_test.csv')
y_train=pd.read_csv('../datasets/y_train_cleaned.csv')

In [3]:
#Debugging code 
Z_test_data = Z_test_data.replace((np.inf, -np.inf, np.nan), 0).reset_index(drop=True)

In [4]:
Z_train.head()

,index,id,pid,lot_frontage,lot_area,lot_shape,utilities,land_slope,overall_qual,overall_cond,...,misc_feature_Othr,misc_feature_Shed,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
0,1957,239,905452110,-0.921358,-0.106446,0.718504,0.0,0.218298,-1.478840,0.389398,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
1,1966,2443,528315030,0.643738,-0.089214,0.718504,0.0,0.218298,1.337753,-0.505889,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
2,838,441,528120170,0.399191,0.150689,-1.039629,0.0,0.218298,2.041901,-0.505889,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
3,427,1379,905103030,0.545919,0.512571,0.718504,0.0,0.218298,-0.070544,-0.505889,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
4,1489,1218,534428020,-0.040992,0.424550,-1.039629,0.0,0.218298,-1.478840,-0.505889,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061


In [5]:
Z_train=Z_train.set_index('index')

In [6]:
Z_train.head()

,id,pid,lot_frontage,lot_area,lot_shape,utilities,land_slope,overall_qual,overall_cond,year_remod/add,...,misc_feature_Othr,misc_feature_Shed,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
index,,,,,,,,,,,,,,,,,,,,,
1957,239,905452110,-0.921358,-0.106446,0.718504,0.0,0.218298,-1.478840,0.389398,-0.268166,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
1966,2443,528315030,0.643738,-0.089214,0.718504,0.0,0.218298,1.337753,-0.505889,0.644691,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
838,441,528120170,0.399191,0.150689,-1.039629,0.0,0.218298,2.041901,-0.505889,1.029051,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
427,1379,905103030,0.545919,0.512571,0.718504,0.0,0.218298,-0.070544,-0.505889,-0.316211,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
1489,1218,534428020,-0.040992,0.424550,-1.039629,0.0,0.218298,-1.478840,-0.505889,-1.181023,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061


In [7]:
Z_train.shape

(1637, 164)

In [8]:
y_train.shape

(1637, 2)

In [9]:
y_train.head()

,Unnamed: 0,sale_price
0,1957,127000
1,1966,348000
2,838,300000
3,427,157500
4,1489,141000


In [10]:
y_train.rename(columns={"Unnamed: 0": "index"}, inplace=True)
y_train=y_train.set_index('index')

In [11]:
y_train.head(2)

,sale_price
index,
1957,127000
1966,348000


In [12]:
Z_test_data.head()

,Unnamed: 0,id,pid,lot_frontage,lot_area,lot_shape,utilities,land_slope,overall_qual,overall_cond,...,misc_feature_Othr,misc_feature_Shed,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
0,0,2658,902301120,0.007918,-0.141587,0.718504,0.0,0.218298,-0.070544,2.179972,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
1,1,2718,905108090,-0.040992,-0.053735,-1.039629,0.0,0.218298,-0.774692,-1.401176,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
2,2,2414,528218130,-0.530084,1.203559,-1.039629,0.0,0.218298,0.633604,-0.505889,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,3.349092,-0.049492,-2.531253
3,3,1989,902207150,-0.432266,-0.246671,0.718504,0.0,0.218298,-0.774692,0.389398,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
4,4,625,535105100,-0.040992,-0.081105,-1.039629,0.0,0.218298,-0.070544,-0.505889,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061


In [13]:
Z_test_data.rename(columns={"Unnamed: 0": "index"}, inplace=True)
Z_test_data=Z_test_data.set_index('index')

In [14]:
Z_test_data.head()

,id,pid,lot_frontage,lot_area,lot_shape,utilities,land_slope,overall_qual,overall_cond,year_remod/add,...,misc_feature_Othr,misc_feature_Shed,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
index,,,,,,,,,,,,,,,,,,,,,
0,2658,902301120,0.007918,-0.141587,0.718504,0.0,0.218298,-0.070544,2.179972,-1.661474,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
1,2718,905108090,-0.040992,-0.053735,-1.039629,0.0,0.218298,-0.774692,-1.401176,-0.364256,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
2,2414,528218130,-0.530084,1.203559,-1.039629,0.0,0.218298,0.633604,-0.505889,1.029051,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,3.349092,-0.049492,-2.531253
3,1989,902207150,-0.432266,-0.246671,0.718504,0.0,0.218298,-0.774692,0.389398,1.029051,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
4,625,535105100,-0.040992,-0.081105,-1.039629,0.0,0.218298,-0.070544,-0.505889,-1.036888,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061


In [15]:
#Renaming unnamed 0 and setting it to the index
Z_test_data.rename(columns={"Unnamed: 0": "index"}, inplace=True)

In [16]:
Z_test_data.head(2)

,id,pid,lot_frontage,lot_area,lot_shape,utilities,land_slope,overall_qual,overall_cond,year_remod/add,...,misc_feature_Othr,misc_feature_Shed,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
index,,,,,,,,,,,,,,,,,,,,,
0,2658,902301120,0.007918,-0.141587,0.718504,0.0,0.218298,-0.070544,2.179972,-1.661474,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
1,2718,905108090,-0.040992,-0.053735,-1.039629,0.0,0.218298,-0.774692,-1.401176,-0.364256,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061


In [17]:
Z_test_data.head()

,id,pid,lot_frontage,lot_area,lot_shape,utilities,land_slope,overall_qual,overall_cond,year_remod/add,...,misc_feature_Othr,misc_feature_Shed,sale_type_CWD,sale_type_Con,sale_type_ConLD,sale_type_ConLI,sale_type_ConLw,sale_type_New,sale_type_Oth,sale_type_WD
index,,,,,,,,,,,,,,,,,,,,,
0,2658,902301120,0.007918,-0.141587,0.718504,0.0,0.218298,-0.070544,2.179972,-1.661474,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
1,2718,905108090,-0.040992,-0.053735,-1.039629,0.0,0.218298,-0.774692,-1.401176,-0.364256,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
2,2414,528218130,-0.530084,1.203559,-1.039629,0.0,0.218298,0.633604,-0.505889,1.029051,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,3.349092,-0.049492,-2.531253
3,1989,902207150,-0.432266,-0.246671,0.718504,0.0,0.218298,-0.774692,0.389398,1.029051,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061
4,625,535105100,-0.040992,-0.081105,-1.039629,0.0,0.218298,-0.070544,-0.505889,-1.036888,...,-0.042848,-0.14781,-0.065532,-0.034975,-0.08947,-0.055351,-0.049492,-0.298588,-0.049492,0.395061


In [18]:
#Dropping Utilities because scaled stange 
Z_train.drop(columns='utilities', inplace=True)
Z_test_data.drop(columns='utilities', inplace=True)

In [19]:
Z_test_data_id=Z_test_data['id']

In [20]:
Z_test_data_id.head(2)

index
0    2658
1    2718
Name: id, dtype: int64

In [21]:
#Dropping id and PID for the model 
Z_train=Z_train.drop(columns=['id', 'pid'])
Z_test_data=Z_test_data.drop(columns=['id', 'pid'])

In [22]:
#Instantiate Best Model 
lr=LinearRegression()

In [23]:
y_train.mean()

sale_price    181428.810629
dtype: float64

In [24]:
#fit best model 
lr.fit(Z_train, y_train)

LinearRegression()

In [25]:
#Train score
lr.score(Z_train, y_train)

0.9260161478331302

In [26]:
#Test data predictions 
lr.predict(Z_test_data)[:10]

array([[136442.89700431],
       [149311.39700431],
       [219489.14700431],
       [109502.64700431],
       [176698.64700431],
       [ 81676.14700431],
       [103176.14700431],
       [153353.14700431],
       [174791.39700431],
       [171163.89700431]])

In [27]:
#Creating an empty dataframe 
kaggle=pd.DataFrame()

In [28]:
#Making preds into a Dataframe 
kaggle['Id']=Z_test_data_id

In [29]:
#Creating a Sale Price column 
kaggle['SalePrice']=lr.predict(Z_test_data)

In [30]:
#Check 
kaggle.head()

,Id,SalePrice
index,,
0,2658,136442.897004
1,2718,149311.397004
2,2414,219489.147004
3,1989,109502.647004
4,625,176698.647004


In [31]:
#Check 
kaggle.tail()

,Id,SalePrice
index,,
873,1662,202681.397004
874,1234,223856.647004
875,1373,118805.397004
876,1672,104844.397004
877,1939,113981.147004


#### The below code is not best practice for this assignment but necessary due to issues with kaggle submission

In [34]:
sorted_df = kaggle.sort_values(by=['Id'], ascending=True)

In [37]:
sorted_df = sorted_df.sort_index(ascending=True)

In [39]:
sorted_df=sorted_df.reset_index().drop(columns='Id')
sorted_df

,index,SalePrice
0,0,136442.897004
1,1,149311.397004
2,2,219489.147004
3,3,109502.647004
4,4,176698.647004
...,...,...
873,873,202681.397004
874,874,223856.647004
875,875,118805.397004
876,876,104844.397004


In [50]:
kaggle=sorted_df.rename(columns={'index':'Id', 'SalePrice':'sample_soln'})

In [51]:
kaggle=kaggle[:260]

In [52]:
kaggle

,Id,sample_soln
0,0,136442.897004
1,1,149311.397004
2,2,219489.147004
3,3,109502.647004
4,4,176698.647004
...,...,...
255,255,110258.397004
256,256,76778.147004
257,257,418435.397004
258,258,209466.147004


In [53]:
#Saving kaggle as a csv
kaggle.to_csv('submission_lr.csv', index=False)